In [2]:
from src import data
import transformers
import numpy as np
import os

# change cwd to project root
while os.path.split(os.getcwd())[-1] != "project":
    os.chdir("..")

NER, SENTIMENT_TOPIC = data.load()

sentences = SENTIMENT_TOPIC["sentence"].to_list()
true_labels = SENTIMENT_TOPIC["topic"].to_list()

ModuleNotFoundError: No module named 'src'

In [28]:

classifier = transformers.pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
candidate_labels = ["sports","book", "movie"]
results = classifier(sentences, candidate_labels)

Device set to use mps:0


In [29]:
print(len(sentences))
print(len(results))
print(len(list(enumerate(results))) )

correct_predictions = 0
for i, result in enumerate(results):
    labels = result["labels"]
    scores = result["scores"]

    predicted_label = labels[np.argmax(scores)]
    true_label = true_labels[i]
    print(f"Sentence {i + 1}: {sentences[i]}")
    print(f"P: {predicted_label}, T: {true_label}")

    if predicted_label == true_label:
        correct_predictions += 1

print(f"Correct prediction proportion: {correct_predictions / len(results)}")


18
18
18
Sentence 1: The stadium was alive with the roar of the crowd after that incredible win.
P: sports, T: sports
Sentence 2: That last-minute goal had me jumping out of my seat—what an unbelievable finish!
P: sports, T: sports
Sentence 3: I couldn’t put the book down; it swept me into a whole new world.
P: book, T: book
Sentence 4: The story had its moments, though some parts felt like they dragged on a bit.
P: movie, T: book
Sentence 5: I enjoyed the way the timelines shifted, even if it got a little confusing sometimes.
P: movie, T: book
Sentence 6: Every time I watch this movie, I notice something new—it really grows on you.
P: movie, T: movie
Sentence 7: The movie was a wild ride from start to finish; I was on the edge of my seat the entire time.
P: movie, T: movie
Sentence 8: Honestly, the team just fell apart in the second half, and it showed on the field.
P: sports, T: sports
Sentence 9: No surprise there—the coach rested a few players given how packed the schedule was.
P: 